In [ ]:
# this file is try to cross validate between data sources

<h1>Load all dependencies</h1>

In [1]:
import os, ast, json, uuid, re
from itertools import combinations
from scipy import stats
import numpy as np
import pandas as pd
import geopandas

import lxml
from lxml import etree
from lxml.etree import QName
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.colors import n_colors
import plotly.io as pio
pio.renderers.default = "browser"
print(pio.renderers)

Renderers configuration
-----------------------
    Default renderer: 'browser'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [2]:
pd.set_option('display.max_columns', None)

In [3]:
xml_path = 'D:/Onedrive/SynMAR2020/แบ่งเขต.xml'
csv_path = "C:/Users/ASUS/Downloads/dist-rep-wran-ampcode-v4.csv"
wspre_csv_path = "C:/Users/ASUS/Downloads/ws_pre_license.csv"

In [4]:
dist_df = pd.read_csv(csv_path)

In [5]:
dist_df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1, inplace=True)

In [6]:
ws_df = pd.read_csv(wspre_csv_path)

In [7]:
ws_df

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
0,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บ้านเลขที่ 598 ชั้น 12 อาคารคิวเฮ้าส์ เพลินจิ...,598,NaN,ชั้น 12,อาคารคิวเฮ้าส์ เพลินจิต,NaN,เพลินจิต,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2617 5000,0 2627 3251,นางสาว นลิน เจนวิทย์วิชัยกุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,0.0,NaN,อนุมัติ,E
1,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บ้านเลขที่ 598 ชั้น 12 อาคารคิวเฮ้าส์ เพลินจิ...,598,NaN,ชั้น 12,อาคารคิวเฮ้าส์ เพลินจิต,NaN,เพลินจิต,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2617 5000,0 2627 3251,นางสาว นลิน เจนวิทย์วิชัยกุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,0.0,NaN,อนุมัติ,E
2,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บริษัท บี.บราวน์ (ประเทศไทย) จำกัด,บ้านเลขที่ 598 ชั้น 12 อาคารคิวเฮ้าส์ เพลินจิ...,598,NaN,ชั้น 12,อาคารคิวเฮ้าส์ เพลินจิต,NaN,เพลินจิต,NaN,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330.0,0 2617 5000,0 2627 3251,นางสาว นลิน เจนวิทย์วิชัยกุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,0.0,NaN,อนุมัติ,E
3,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ3 สถานพยาบาล เอกชน,5902038.0,283/2560 (วจ.3),อนุมัติ,NaN
4,บริษัท อลินดา เอสเธททิคซ์ จำกัด,อลินดาคลินิกเวชกรรม,บ้านเลขที่865-867 ซอย ถนน สุขุมวิท หมู่ ตำบ...,865-867,NaN,NaN,NaN,NaN,สุขุมวิท,NaN,คลองตันเหนือ,วัฒนา,กรุงเทพมหานคร,10110.0,"0 2106 3689,06 1416 0505",NaN,นาย นวโภค เจริญระบิน,ครอบครอง วจ4 สถานพยาบาล เอกชน,5902037.0,435/2560 (วจ.4),อนุมัติ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,นางสาว กาญจน์กมล ชามะรัตน์,ลักซ์ บางกอก คลินิกเวชกรรม,บ้านเลขที่236/2 ชั้น 1 ถนนนาคนิวาส ตำบลลาด...,236/2,NaN,ชั้น 1,NaN,NaN,นาคนิวาส,NaN,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,10230.0,0 2000 3732,NaN,นางสาว กาญจน์กมล ชามะรัตน์,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800068.0,ควจ4-ร 18/2568,อนุมัติ,NaN
3881,นาย อภิวัฒน์ บรรเทา,นิวยูคลินิกเฉพาะทางด้านเวชกรรมจิตเวช,บ้านเลขที่1105 ห้องเลขที่ R.9 ชั้น 2 โครงการ S...,1105,ห้องเลขที่ R.9,ชั้น 2,โครงการ S PLACE COMMUNITY MALL อาคาร 3,NaN,พัฒนาการ,NaN,ประเวศ,ประเวศ,กรุงเทพมหานคร,10250.0,NaN,NaN,นาย อภิวัฒน์ บรรเทา,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800069.0,ควจ4-ร 19/2568,อนุมัติ,NaN
3882,บริษัท โรช ไทยแลนด์ จำกัด,บริษัท โรชไทยแลนด์ จำกัด,บ้านเลขที่89 ชั้น 26 อาคารเอไอเอ แคปปิตอล เซ็...,89,NaN,ชั้น 26,อาคารเอไอเอ แคปปิตอล เซ็นเตอร์,NaN,รัชดาภิเษก,NaN,ดินแดง,ดินแดง,กรุงเทพมหานคร,10400.0,0 2017 5600,NaN,นาย ภูมินทร์ ยั่งยืนนาน,ครอบครอง วจ 4 เพื่อการวิเคราะห์หรือการศึกษาหรื...,6800071.0,ควจ4-ว 1/2568,อนุมัติ,NaN
3883,นาย วิษณุ เจริญสุวรรณ,แพทย์วิษณุคลินิกเวชกรรม,บ้านเลขที่427/22-23 ชั้น 1 ถนนประชาราษฎร์ 2...,427/22-23,NaN,ชั้น 1,NaN,NaN,ประชาราษฎร์ 2,NaN,บางซื่อ,บางซื่อ,กรุงเทพมหานคร,10800.0,08 3747 9889,NaN,นาย วิษณุ เจริญสุวรรณ,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6800079.0,ควจ4-ร 23/2568,อนุมัติ,NaN


<h1> Parsing XML (skip this part, if using csv)</h1>

In [18]:
namespaces = {
    'ss': 'urn:schemas-microsoft-com:office:spreadsheet',
    'x': 'urn:schemas-microsoft-com:office:excel',
    'o': 'urn:schemas-microsoft-com:office:office',
    'html': 'http://www.w3.org/TR/REC-html40',
    '': 'urn:schemas-microsoft-com:office:spreadsheet' # Default namespace
}

In [7]:
tree = etree.parse(xml_path)

In [8]:
root = tree.getroot()

In [12]:
root.findall('.//ss:Worksheet', namespaces)

[<Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a2d6940>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cb80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cdc0>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16ce40>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cd80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16ca80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16da40>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16d240>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cec0>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16cf80>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet at 0x1dc9a16c800>,
 <Element {urn:schemas-microsoft-com:office:spreadsheet}Worksheet

In [19]:

ss_name_qn = QName(namespaces['ss'], 'Name')
ss_row_qn = QName(namespaces['ss'], 'Row')
ss_cell_qn = QName(namespaces['ss'], 'Cell')
ss_data_qn = QName(namespaces['ss'], 'Data')
ss_type_qn = QName(namespaces['ss'], 'Type')

In [16]:
worksheet_names = []
for worksheet in root.findall('.//ss:Worksheet', namespaces):
    name = worksheet.get(ss_name)
    if name:
        worksheet_names.append(name)

In [21]:
namespaces

{'ss': 'urn:schemas-microsoft-com:office:spreadsheet',
 'x': 'urn:schemas-microsoft-com:office:excel',
 'o': 'urn:schemas-microsoft-com:office:office',
 'html': 'http://www.w3.org/TR/REC-html40',
 '': 'urn:schemas-microsoft-com:office:spreadsheet'}

In [25]:
len(root.findall('.//ss:Worksheet', namespaces)[0].find('.//ss:Table', namespaces).findall(ss_row_qn))

3886

In [26]:
first_ws_allrows = root.findall('.//ss:Worksheet', namespaces)[0].find('.//ss:Table', namespaces).findall(ss_row_qn)

In [27]:
table_data = []
for row in first_ws_allrows:
    row_data = []
    # Iterate through all Cell elements in the Row
    for cell in row.findall(ss_cell_qn):
        # Find the Data element within the Cell
        data_element = cell.find(ss_data_qn)
        if data_element is not None:
            # Get the data type and text content
            # The get() method returns a Unicode string, which is what we want
            data_value = data_element.text
            row_data.append(data_value)
        else:
            # Append an empty string or None for empty cells
            row_data.append(None) 
    table_data.append(row_data)

In [29]:
# Create the pandas DataFrame from the extracted data
df = pd.DataFrame(table_data)

# The first row often contains headers. You can set them as column names.
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

<h1>EDA starts here</h1>

In [8]:
ws_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3885 entries, 0 to 3884
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ชื่อผู้รับอนุญาต     3885 non-null   object 
 1   ชื่อสถานที่          3885 non-null   object 
 2   ที่อยู่สถานที่       3885 non-null   object 
 3   บ้านเลขที่           3872 non-null   object 
 4   ห้อง                 281 non-null    object 
 5   ชั้น                 957 non-null    object 
 6   อาคาร                531 non-null    object 
 7   ซอย                  1314 non-null   object 
 8   ถนน                  3188 non-null   object 
 9   หมู่                 529 non-null    object 
 10  ตำบล                 3881 non-null   object 
 11  อำเภอ                3881 non-null   object 
 12  จังหวัด              3881 non-null   object 
 13  รหสไปรษณีย์          3881 non-null   float64
 14  เบอร์โทรศัพท์        3807 non-null   object 
 15  เบอร์โทรสาร          642 non-null    o

In [10]:
ws_df['ประเภทคำขอ'].value_counts().reset_index()

,ประเภทคำขอ,count
0,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,759
1,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,750
2,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,542
3,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,359
4,ครอบครอง วจ 3 เพื่อการบำบัดรักษาหรือป้องกันโรค...,327
5,ครอบครอง วจ2 สถานพยาบาล เอกชน,149
6,ครอบครอง วจ4 สถานพยาบาล เอกชน,131
7,ครอบครอง วจ3 สถานพยาบาล เอกชน,100
8,นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ใ...,96
9,ครอบครอง วจ 1 เพื่อการวิเคราะห์หรือการศึกษาหรื...,74


In [9]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('ขาย')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-นิติบุคคล)                542
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-บุคคลธรรมดา)              359
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (รัฐ)                             62
ขาย วจ 1 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์      1
Name: count, dtype: int64

In [10]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('จำหน่าย')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว์ป่วยในทางการแพทย์ (หมวด ค.)        759
จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว์ป่วยในทางการแพทย์ (หมวด ก.)         60
จำหน่าย  ยส 2 เพื่อใช้ในการขนส่งสาธารณะ (หมวด ข.)                                  15
จำหน่าย ยส 2 เพื่อการศึกษาวิจัยทางด้านการแพทย์หรือทางด้านวิทยาศาสตร์ (หมวด ก.)      1
Name: count, dtype: int64

In [11]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('วจ 2')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์                                                                  96
ครอบครอง วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์                                                  41
นำเข้า วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์                                                    33
ครอบครอง วจ 2 เพื่อใช้ประจำในการปฐมพยาบาลหรือในกรณีเกิดเหตุฉุกเฉินในยานพาหนะที่ใช้ในการขนส่งสาธารณะระหว่างประเทศที่จดทะเบียนในราชอาณาจักร    29
ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์                                                                    18
ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์                                                      12
ครอบครอง วจ 2 เพื่อป้องกันและปราบปรามการกระทำความผิดเกี่ยวกับวัตถุออกฤทธิ์                                                   

In [16]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('วจ2')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ครอบครอง วจ2 สถานพยาบาล เอกชน          149
ครอบครอง วจ2 ไม่ใช่สถานพยาบาล เอกชน      4
ครอบครอง วจ2 เพื่อใช้ทางวิทยาศาสตร์      1
Name: count, dtype: int64

In [12]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('วัตถุออกฤท')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-นิติบุคคล)      542
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (เอกชน-บุคคลธรรมดา)    359
นำเข้า วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์      96
ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือป้องกันโรค (รัฐ)                   62
ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์        18
ครอบครอง วจ 2 เพื่อป้องกันและปราบปรามการกระทำความผิดเกี่ยวกับวัตถุออกฤทธิ์        2
ส่งออก วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์       1
Name: count, dtype: int64

In [13]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('ผลิต')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
เพื่อผลิต ยส 2                                                                             52
ครองครอง ยส 2 เพื่อผลิต ยส3 (หมวด ค.)                                                      47
ผลิต วจ 2 เพื่อการบริหารจัดการวัตถุออกฤทธิ์ในประเภท 2 ที่ใช้ในทางการแพทย์                  18
ผลิต วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์    12
ครองครอง ยส 2 เพื่อผลิต ยส3 (หมวด ก.)                                                       2
Name: count, dtype: int64

In [15]:
ws_df.loc[ws_df['ประเภทคำขอ'].str.contains('ครอบครอง')]['ประเภทคำขอ'].value_counts()

ประเภทคำขอ
ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรคให้แก้ป่วยหรือสัตว์ป่วยในทางการแพทย์                                                           750
ครอบครอง วจ 3 เพื่อการบำบัดรักษาหรือป้องกันโรคให้แก้ป่วยหรือสัตว์ป่วยในทางการแพทย์                                                           327
ครอบครอง วจ2 สถานพยาบาล เอกชน                                                                                                                149
ครอบครอง วจ4 สถานพยาบาล เอกชน                                                                                                                131
ครอบครอง วจ3 สถานพยาบาล เอกชน                                                                                                                100
ครอบครอง วจ 1 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์                                                   74
ครอบครอง วจ 2 เพื่อการวิเคราะห์หรือการศึกษาหรือเพื่อประโยชน์อื่น ทางการแพทย์หรือวิทยาศาสตร์                            

In [14]:
ws_df['จังหวัด'].value_counts().reset_index()

,จังหวัด,count
0,กรุงเทพมหานคร,3477
1,สมุทรปราการ,74
2,ปทุมธานี,63
3,นนทบุรี,60
4,เชียงใหม่,28
5,ชลบุรี,20
6,สมุทรสาคร,14
7,นครปฐม,12
8,สงขลา,11
9,นครราชสีมา,11


In [54]:
df['เลขใบอนุญาต'].value_counts().reset_index().sort_values(by=['count'],ascending=False)

,เลขใบอนุญาต,count
0,2/2564,5
1,1/2564,5
2,4/2564,4
3,5/2564,4
4,6/2564,4
...,...,...
3358,ควจ4-ร 19/2568,1
3359,ควจ4-ว 1/2568,1
3360,ควจ4-ร 23/2568,1
3361,ควจ4-ร 21/2568,1


In [57]:
df['เลขรับ'].value_counts().reset_index().sort_values(by=['count'],ascending=False)

,เลขรับ,count
0,0,240
7,6800077,1
3608,5900758,1
23,5902038,1
3624,5902037,1
...,...,...
19,6800017,1
20,6800024,1
21,6800025,1
22,6800030,1


In [56]:
df.loc[df['เลขใบอนุญาต'] == '1/2564']

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
1817,นางสาว พรพิศ นคเรศไอศูรย์,เมดพลัสคลินิกเวชกรรม,บ้านเลขที่900/31 ห้องเลขที่ี G5 ชั้น G อาคารเอ...,900/31,ห้องเลขที่ี G5,ชั้น G,อาคารเอสวีซิตี้ อาคาร 12,None,พระราม 3,None,บางโพงพาง,ยานนาวา,กรุงเทพมหานคร,10120,06 4556 5424,None,นางสาว พรพิศ นคเรศไอศูรย์,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6400020,1/2564,อนุมัติ,None
1847,บริษัท โรงพยาบาล ซีจีเอช ลำลูกกา จำกัด,โรงพยาบาลซีจีเอช ลำลูกกา โรงพยาบาลทั่วไปขนาดใหญ่,บ้านเลขที่80/77-80 ถนน หมู่5 ตำบลลำลูกกา อ...,80/77-80,None,None,None,None,None,5,ลำลูกกา,ลำลูกกา,ปทุมธานี,12150,02 088 0888,02 088 0899,พลอากาศโท วันชัย ศิริเสรีวรรณ,จำหน่าย ยส 2 เพื่อการบำบัดรักษาผู้ป่วยหรือสัตว...,6400392,1/2564,อนุมัติ,None
1979,บริษัท พีลาทัส มารีน จำกัด,บริษัท พีลาทัส มารีน จำกัด,บ้านเลขที่84/1-4 ชั้น 1-4 อาคารยังเพลส แกรนด์...,84/1-4,None,ชั้น 1-4,อาคารยังเพลส แกรนด์ เลอ จาแด๊งน์,พหลโยธิน 37,None,None,ลาดยาว,จตุจักร,กรุงเทพมหานคร,10900,0 2930 1801-9,None,นาย วราวิช ฉิมตะวัน,จำหน่าย ยส 2 เพื่อใช้ในการขนส่งสาธารณะ (หมวด ข.),6400812,1/2564,อนุมัติ,None
2004,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บริษัท มุนดิฟาร์มา (ประเทศไทย) จำกัด,บ้านเลขที่127 ห้อง 25.02 ชั้น 25 อาคาร เกษรทาว...,127,ห้อง 25.02,ชั้น 25,อาคาร เกษรทาวเวอร์,None,ราชดำริ,None,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330,0 2073 0224,None,นางสาว สุทัชชา อัจฉริยวงศ์กุล,ครอบครอง ยส 2 เพื่อประโยชน์ทางราชการ หมวด ค,6400866,1/2564,อนุมัติ,None
2010,องค์การเภสัชกรรม,องค์การเภสัชกรรม,บ้านเลขที่ 75/1 ซอย ถนนพระราม 6 หมู่ ตำบลทุ...,75/1,None,None,None,None,พระราม 6,None,ทุ่งพญาไท,ราชเทวี,กรุงเทพมหานคร,10400,0 2203 8600-3,None,นาย วิฑูรย์ ด่านวิบูลย์,เพื่อผลิต ยส 2,6400868,1/2564,อนุมัติ,None


In [63]:
dist_df

,ReportDisburseNarcoticBeYear,ReportDisburseNarcoticMonth,ProductTypeName,MonthlyReportNarcoticName,MonthlyReportNarcoticBalanceForwardValue,MonthlyReportNarcoticReceiveValue,MonthlyReportNarcoticPaidValue,MonthlyReportNarcoticRemainValue,ReferCustomerName,CustomerAddressProvinceName,CustomerTypeName,ReportDisburseNarcoticRecordType,CreateDateTime,ReportDisburseNarcoticRequestorName,SendReportDate,uuid_x,uuid_y,fullAddress,formatted_address,location_type,place_id,lat,lng,types,zip_code,district_name,amp_code
0,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),1829.0,0.0,500.0,1329.0,โรงพยาบาล ศรีประจันต์,สุพรรณบุรี,สถานพยาบาลของรัฐ,RECORD,16/03/2024 14:43 น.,นางสาว สมัชญา บุญขวัญดี,3/16/2024,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,ddb6e8dd-53a5-44c4-b727-df5d5884e7f8,218 ถนนสุพรรณบุรี-ชัยนาถ แขวงวังน้ำซับ เขตศรีป...,"218 สุพรรณบุรี - ชัยนาท Tambon Si Prachan, Amp...",ROOFTOP,ChIJ7dkWP78k4jAR5xZD6aeOngA,14.641639,100.149439,"premise,street_address",72140,ศรีประจันต์,7205
1,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),500.0,0.0,0.0,500.0,คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช,ภูเก็ต,คลินิก,RECORD,14/03/2024 9:08 น.,นางสาว จริยา บัวศรี,3/14/2024,efd5c13a-af08-46be-8591-843fec8306a9,82862aaa-36cd-40fc-9c51-222b2401a4b5,47/14-15 ถนนราษฎร์อุทิศ 200 ปี ตำบลป่าตอง อำเภ...,"Rat Uthit Song Roi Pi Rd, Tambon Patong, Ampho...",GEOMETRIC_CENTER,ChIJc_7cxrw6UDARWxKMiwN1tDQ,7.894495,98.298619,route,83150,กะทู้,8302
2,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),1889.0,0.0,612.0,1277.0,โรงพยาบาลซานคามิลโลโรงพยาบาลทั่วไปขนาดใหญ่,ราชบุรี,โรงพยาบาลเอกชน,RECORD,28/02/2024 13:12 น.,นางสาว จริยา บัวศรี,2/28/2024,aa40eacf-4e16-489f-bf4b-bba34bd86944,15666bbc-9341-40ae-b42b-c489288a7503,31 ถนนอุดมพิทยา ตำบลสวนกล้วย อำเภอบ้านโป่ง ราช...,"31 Soi Udum Pitthaya Khang Rong Rian Hok Heng,...",RANGE_INTERPOLATED,EnUzMSBTb2kgVWR1bSBQaXR0aGF5YSBLaGFuZyBSb25nIF...,13.813480,99.875546,street_address,70110,บ้านโป่ง,7005
3,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.25 mg (Polipharm),310.0,2000.0,465.0,1845.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,e5062c3e-861b-44df-9fd6-459e62272db1,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
4,2567.0,1.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablets 0.5 mg (Polipharm),754.0,3000.0,957.0,797.0,โรงพยาบาล บางปะกอก 3 โรงพยาบาลทั่วไปขนาดใหญ่,สมุทรปราการ,โรงพยาบาลเอกชน,RECORD,16/02/2024 9:24 น.,นางสาว จริยา บัวศรี,2/16/2024,f84ad883-2bea-4b36-ae4d-0c0059ee6697,e5062c3e-861b-44df-9fd6-459e62272db1,27/14 หมู่ 10 ถนนสุขสวัสดิ์ แขวงบางครุ เขตพระป...,"Thanon Suk Sawat, Thailand",GEOMETRIC_CENTER,ChIJ73Yei16h4jARTTUpslTDs5A,13.622237,100.539377,route,10130,พระประแดง,1104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28174,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,9551310e-ba2a-48c8-8f68-14f680585379,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxszERwF2-kBj0C_A,6.412776,101.823887,street_address,96000,เมืองนราธิวาส,9601
28175,2567.0,12.0,(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2,Alprazolam tablet 1.0 mg (Polipharm) (100 tab/...,949.0,0.0,342.0,607.0,หมอกิ๊ฟลันสหคลินิก,นราธิวาส,คลินิก,ONLINE,10/01/2025 15:26 น.,นางสาว อารีนา หะยีวาเงาะ,1/10/2025,2ae8194a-7022-4837-88b3-9baaab86a58e,d5ca07fe-e5b9-46b9-aeae-e2b3a69552d7,354/23 ถนนระแงะมรรคา ตำบลบางนาค อำเภอเมืองนราธ...,"354 Thanon Rangae Makkha, Tambon Bang Nak, Amp...",ROOFTOP,ChIJ8aQTNLzxsz

In [68]:
loc_ref_pvhos = dist_df.loc[dist_df['CustomerTypeName'] == 'คลินิก']['ReferCustomerName'].drop_duplicates()

In [77]:
dist_df['CustomerAddressProvinceName'].value_counts()

CustomerAddressProvinceName
กรุงเทพมหานคร    9243
ชลบุรี           1271
นนทบุรี          1208
เชียงใหม่        1062
สมุทรปราการ       874
                 ... 
สุโขทัย            48
ยโสธร              36
มหาสารคาม          35
กาฬสินธุ์          32
สตูล               12
Name: count, Length: 76, dtype: int64

In [78]:
loc_ref_pvhos_bkk = dist_df.loc[(dist_df['CustomerTypeName'] == 'คลินิก') & (dist_df['CustomerAddressProvinceName'] == 'กรุงเทพมหานคร')]['ReferCustomerName'].drop_duplicates()

In [79]:
loc_ref_pvhos_bkk

23                                แอ็บส์เมดิคคลินิกเวชกรรม
30       เพื่อนใจคลินิกเฉพาะทางด้านเวชกรรมจิตเวชเด็กและ...
44                               สหแพทย์รัชดาคลินิกเวชกรรม
58                                       มณฑลคลินิกเวชกรรม
65                              คลินิกเวชกรรมแพทย์ลาดพร้าว
                               ...                        
26138                                    ปีติคลินิกเวชกรรม
26262                                    ไวทัลไลฟ์สหคลินิก
26487                          คลินิกเวชกรรมดุสิตาการแพทย์
26840              บ้านสาทรคลินิกเฉพาะทางด้านเวชกรรมจิตเวช
27443               ดีจีบีสหคลินิก (เวชกรรมและกายภาพบำบัด)
Name: ReferCustomerName, Length: 226, dtype: object

In [69]:
loc_ref_pvhos

1                        คลินิกเวชกรรมด็อกเตอร์ป่าตองวรเวช
5                           คลินิกเวชกรรมแพทย์หญิงกนกนันท์
6                                    อนุรักษ์คลินิกเวชกรรม
17                                     สีชมพูคลินิกเวชกรรม
21       เอส ไมนด์ คลินิกเฉพาะทางด้านเวชกรรมสาขาจิตเวชศ...
                               ...                        
26840              บ้านสาทรคลินิกเฉพาะทางด้านเวชกรรมจิตเวช
26898           คลินิกเฉพาะทางด้านเวชกรรมจิตเวชแฮปปี้มายด์
27443               ดีจีบีสหคลินิก (เวชกรรมและกายภาพบำบัด)
28028                             คลินิกเวชกรรมดิเรก-พรรณี
28172                                   หมอกิ๊ฟลันสหคลินิก
Name: ReferCustomerName, Length: 611, dtype: object

In [80]:
isPresent = []
for name in loc_ref_pvhos_bkk:
    loc_sitename = df.loc[df['ชื่อสถานที่'].str.contains(name)]
    if len(loc_sitename) >0:
        isPresent.append(True)
        continue
    loc_mahname = df.loc[df['ชื่อผู้รับอนุญาต'].str.contains(name)]
    if len(loc_mahname) >0:
        isPresent.append(True)
        continue
    else:
        isPresent.append(False)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_30652\853211432.py:3: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.

C:\Users\ASUS\AppData\Local\Temp\ipykernel_30652\853211432.py:7: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.



In [72]:
ref_pvhos_df = pd.DataFrame(loc_ref_pvhos)

In [81]:
ref_pvhos_bkk_df = pd.DataFrame(loc_ref_pvhos_bkk)

In [73]:
ref_pvhos_df['isPresent'] = isPresent

In [82]:
ref_pvhos_bkk_df['isPresent'] = isPresent

In [85]:
ref_pvhos_bkk_df.loc[ref_pvhos_bkk_df['isPresent'] == False].loc[599]

ReferCustomerName    ่คลินิกเวชกรรมแพทย์อิทธิพล
isPresent                                 False
Name: 599, dtype: object

In [86]:
df.loc[df['ชื่อสถานที่'].str.contains('แพทย์อิทธิพล')]

,ชื่อผู้รับอนุญาต,ชื่อสถานที่,ที่อยู่สถานที่,บ้านเลขที่,ห้อง,ชั้น,อาคาร,ซอย,ถนน,หมู่,ตำบล,อำเภอ,จังหวัด,รหสไปรษณีย์,เบอร์โทรศัพท์,เบอร์โทรสาร,ชื่อผู้ดำเนินกิจการ,ประเภทคำขอ,เลขรับ,เลขใบอนุญาต,สถานะ,รูปแบบ
914,นาย อิทธิพล มกรเสน,คลินิกเวชกรรมแพทย์อิทธิพล,บ้านเลขที่519/17 ซอย ถนนประชาอุทิศ หมู่ ตำบ...,519/17,None,None,None,None,ประชาอุทิศ,None,ทุ่งครุ,ทุ่งครุ,กรุงเทพมหานคร,10140,"0 2815 8825, 0 2815 9416",None,นาย อิทธิพล มกรเสน,ขายวัตถุออกฤทธิ์ในประเภท 2 เพื่อบำบัดรักษาหรือ...,6100776,625/2561,อนุมัติ,None
1919,นาย อิทธิพล มกรเสน,คลินิกเวชกรรมแพทย์อิทธิพล,บ้านเลขที่519/17 ซอย ถนนประชาอุทิศ หมู่ ตำบ...,519/17,None,None,None,None,ประชาอุทิศ,None,ทุ่งครุ,ทุ่งครุ,กรุงเทพมหานคร,10140,"0 2815 8825, 0 2815 9416",None,นาย อิทธิพล มกรเสน,ครอบครอง วจ 4 เพื่อการบำบัดรักษาหรือป้องกันโรค...,6400652,ควจ4-ร 14/2564,อนุมัติ,None


In [87]:
df.to_csv("C:/Users/ASUS/Downloads/ws_pre_license.csv", index=False)